### 1) Optical Flow

#### optical flow analysis has some assumptions:
#### 1) the pixel intensities of an object do not change between consecutive frames
#### 2) neighboring pixels have similar motion  

#### using opencv we pass in the previous frame, previous points and the current frame to Lucas-Kanade function  

#### this function attempts to locate the points in the current frame 

In [4]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
corner_track_params = dict(maxCorners = 10,qualityLevel = 0.3,minDistance = 7,blockSize = 7)

In [3]:
lk_params = dict(winSize = (200,200),maxLevel = 2 , criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT , 10 ,0.03))

In [6]:
cap = cv2.VideoCapture(0)

ret , prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame , cv2.COLOR_BGR2GRAY)


# points to track

prevPoints = cv2.goodFeaturesToTrack(prev_gray,mask = None ,**corner_track_params)

mask = np.zeros_like(prev_frame)


while True:
    
    ret , frame = cap.read()
    
    
    frame_gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    
    nextPts , status , err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPoints,None,**lk_params)
    
    
    good_new = nextPts[status == 1]
    good_prev = prevPoints[status == 1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev , y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new) , (x_prev,y_prev),(0,255,0),3)
        
        frame = cv2.circle(frame , (x_new,y_new) , 8,(0,0,255),-1)
        
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    
    if k == 27 :
        break
        
    prev_gray = frame_gray.copy()
    prevPoints = good_new.reshape(-1,1,2)

    
    

cv2.destroyAllWindows() 
cap.release()


### 2) DENSE OPTICAL FLOW

In [2]:
cap = cv2.VideoCapture(0)

ret , frame1 = cap.read()

prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    
    ret , frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0) # default params
    
    mag , ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag , None ,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)
    
    k = cv2.waitKey(10) & 0xFF
    if k ==27:
        break
        
    prvsImg = nextImg
cap.release()
cv2.destroyAllWindows()
    

    

### 3) MeanShift and CamShift 

In [2]:
cap = cv2.VideoCapture(0)

ret , frame = cap.read()


# we will detect the face at first then we will make the meanshift algorithm track it 

face_cascade = cv2.CascadeClassifier('DATA/haarcascades/haarcascade_frontalface_default.xml')

face_rects = face_cascade.detectMultiScale(frame)

(face_x , face_y ,w ,h) = tuple(face_rects[0])

track_window = (face_x , face_y ,w ,h)

roi = frame[face_y:face_y+h,face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi , cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    
    ret , frame = cap.read()
    
    if ret == True :
        
        hsv = cv2.cvtColor(frame , cv2.COLOR_BGR2HSV)
        
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        ########USING MEANSHIFT#############
        # here we notice that the size of the rectangle is not with changeable size so we will use camshift
        #ret , track_window = cv2.meanShift(dst,track_window,term_crit)
        
        #x,y,w,h = track_window
        
        #img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        ###################################
        
        ##########USING CAMSHIFT###########
        ret , track_window = cv2.CamShift(dst,track_window,term_crit)
        
        pts = cv2.boxPoints(ret)
        
        pts = np.int0(pts)
        
        img2 = cv2.polylines(frame,[pts],True ,(0,0,255),5)
        ###################################
        
        
        
        cv2.imshow('img' , img2)
        
        k= cv2.waitKey(1) & 0xFF
        if k == 27:
            break
            
    else:
        break
        
        
cv2.destroyAllWindows()
cap.release()

### 4) Tracking APIs

In [2]:
def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()


    return tracker
 

In [4]:
tracker = ask_for_tracker()


Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  4


In [5]:
str(tracker).split()[0][1:]

'TrackerMedianFlow'

In [6]:
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

# Read video
cap = cv2.VideoCapture(0)

# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
roi = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()

cv2.destroyAllWindows()


Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  0
